In [22]:
import requests
import pandas as pd

In [23]:
page = requests.get("https://anfp.cl/estadisticas")
page

<Response [200]>

In [21]:
#page.content

In [24]:
from bs4 import BeautifulSoup

In [25]:
soup = BeautifulSoup(page.content, 'html.parser')


### Si la página contiene una tabla, podemos encontrar usando soup.find

In [42]:
table_anfp=soup.find('table', { 'class' : 'table tbls table-striped table-bordered dt-responsive display nowrap' })

In [43]:
table_anfp

<table cellspacing="0" class="table tbls table-striped table-bordered dt-responsive display nowrap" id="datatable-responsive" width="100%">
<thead>
<tr>
<th>POS</th>
<th>CLUB</th>
<th>PTS</th>
<th>PJ</th>
<th>PG</th>
<th>PE</th>
<th>PP</th>
<th>GF</th>
<th>GC</th>
<th>DIF</th>
</tr>
</thead>
<tbody>
<tr>
<td class="" style="font-size:25px; font-weight:bold; ">1</td>
<td> <a href="/club/47/universidad-catlica">
<!--<span style="float:left; font-size:23px; margin-left:-15px"> * </span> --> <img src="https://anfp.cl/imagenes/clubes/d01105ee6df81f19d3d25c2736c7d6ae.png" style="margin-right:10px" width="35"/> <span class="tiClub">Universidad Católica</span>
</a></td>
<td>18</td>
<td>9</td>
<td>6</td>
<td>0</td>
<td>3</td>
<td>13</td>
<td>9</td>
<td>4</td>
</tr>
<tr>
<td class="" style="font-size:25px; font-weight:bold; ">2</td>
<td> <a href="/club/52/unin-la-calera">
<!--<span style="float:left; font-size:23px; margin-left:-15px"> * </span> --> <img src="https://anfp.cl/imagenes/clubes/9012

### En la tabla:
- $<th>$ es header de la tabla
- $<tr>$ es una fila de la tabla
- $<td>$ son los datos en la fila

Queremos extraer el header y los datos por cada fila

In [44]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [46]:
list_table = tableDataText(table_anfp)
list_table

[['POS', 'CLUB', 'PTS', 'PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'DIF'],
 ['1', 'Universidad Católica', '18', '9', '6', '0', '3', '13', '9', '4'],
 ['2', 'Unión La Calera', '17', '9', '5', '2', '2', '8', '7', '1'],
 ['3', 'Ñublense', '16', '10', '5', '1', '4', '17', '12', '5'],
 ['4', 'Audax Italiano', '16', '9', '4', '4', '1', '8', '5', '3'],
 ['5', 'Colo Colo', '16', '9', '5', '1', '3', '12', '10', '2'],
 ['6', "O'Higgins", '16', '10', '4', '4', '2', '8', '9', '-1'],
 ['7', 'Deportes La Serena', '15', '10', '4', '3', '3', '13', '7', '6'],
 ['8', 'Unión Española', '15', '10', '4', '3', '3', '15', '12', '3'],
 ['9', 'Deportes Melipilla', '14', '10', '4', '2', '4', '13', '12', '1'],
 ['10', 'Everton', '13', '9', '3', '4', '2', '7', '10', '-3'],
 ['11', 'Deportes Antofagasta', '12', '9', '3', '3', '3', '13', '12', '1'],
 ['12', 'Universidad de Chile', '12', '9', '3', '3', '3', '7', '7', '0'],
 ['13', 'Cobresal', '11', '10', '3', '2', '5', '10', '11', '-1'],
 ['14', 'Palestino', '8', '7', '2',

Lo guardamos en un dataframe de pandas

In [48]:
df_anfp=pd.DataFrame(list_table[1:], columns=list_table[0])
df_anfp.head()

,POS,CLUB,PTS,PJ,PG,PE,PP,GF,GC,DIF
0,1,Universidad Católica,18,9,6,0,3,13,9,4
1,2,Unión La Calera,17,9,5,2,2,8,7,1
2,3,Ñublense,16,10,5,1,4,17,12,5
3,4,Audax Italiano,16,9,4,4,1,8,5,3
4,5,Colo Colo,16,9,5,1,3,12,10,2


### Otra forma fácil de hacerlo es con read_html de pandas
El comando busca directamente la clase 'table' en la página e identifica las entradas $<th>$, $<tr>$ y $<td>$

In [29]:
df_list=pd.read_html(page.text) 

In [30]:
df = df_list[0]

In [31]:
df.head()

,POS,CLUB,PTS,PJ,PG,PE,PP,GF,GC,DIF
0,1,Universidad Católica,18,9,6,0,3,13,9,4
1,2,Unión La Calera,17,9,5,2,2,8,7,1
2,3,Ñublense,16,10,5,1,4,17,12,5
3,4,Audax Italiano,16,9,4,4,1,8,5,3
4,5,Colo Colo,16,9,5,1,3,12,10,2
